# Projeto PLN

Notebook para segunda e terceira partes do projeto.

Datasets: http://storage.googleapis.com/books/ngrams/books/datasetsv2.html (Ainda não entendi como usar as informações dessa página.)




### Importação de dados

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/googlebooks-eng-all-5gram-20090715-100.csv', sep="\t", header=None)

Antes de continuar, vamos verificar se podemos otimizar os dados importados para consumir menos RAM do Google Colab (que é limitado a 12GB por sessão free)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16545658 entries, 0 to 16545657
Data columns (total 5 columns):
0    object
1    int64
2    int64
3    int64
4    int64
dtypes: int64(4), object(1)
memory usage: 631.2+ MB


In [4]:
df.head()

,0,1,2,3,4
0,! ! ! 5 .,1803,1,1,1
1,! ! ! 5 .,1806,1,1,1
2,! ! ! 5 .,1824,1,1,1
3,! ! ! 5 .,1850,1,1,1
4,! ! ! 5 .,1851,1,1,1


In [5]:
df[6000220:6000225]

,0,1,2,3,4
6000220,being the spokesman for the,1963,3,3,3
6000221,being the spokesman for the,1964,1,1,1
6000222,being the spokesman for the,1965,7,7,7
6000223,being the spokesman for the,1966,5,5,5
6000224,being the spokesman for the,1967,1,1,1


In [6]:
df.columns = ['n_grams', 'year', 'match_count', 'page_count', 'volume_count']

In [7]:
df.n_grams = df.n_grams.astype('str')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16545658 entries, 0 to 16545657
Data columns (total 5 columns):
n_grams         object
year            int64
match_count     int64
page_count      int64
volume_count    int64
dtypes: int64(4), object(1)
memory usage: 631.2+ MB


## Data prep

Vamos remover 5-grams que não tenham todos os 5 termos não nulos. Por que? Para poupar memória para os próximos passos.

In [9]:
import re

In [10]:
df['word_count'] = df['n_grams'].apply(lambda sent: len(re.findall(r'\w+', sent)))

In [11]:
df.head()

,n_grams,year,match_count,page_count,volume_count,word_count
0,! ! ! 5 .,1803,1,1,1,1
1,! ! ! 5 .,1806,1,1,1,1
2,! ! ! 5 .,1824,1,1,1,1
3,! ! ! 5 .,1850,1,1,1,1
4,! ! ! 5 .,1851,1,1,1,1


In [12]:
df[6000220:6000225]

,n_grams,year,match_count,page_count,volume_count,word_count
6000220,being the spokesman for the,1963,3,3,3,5
6000221,being the spokesman for the,1964,1,1,1,5
6000222,being the spokesman for the,1965,7,7,7,5
6000223,being the spokesman for the,1966,5,5,5,5
6000224,being the spokesman for the,1967,1,1,1,5


In [13]:
df.where(df.word_count == 5, inplace=True)

In [14]:
df.dropna(inplace=True)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7291145 entries, 5879 to 16545370
Data columns (total 6 columns):
n_grams         object
year            float64
match_count     float64
page_count      float64
volume_count    float64
word_count      float64
dtypes: float64(5), object(1)
memory usage: 389.4+ MB


In [16]:
df.head(5)

,n_grams,year,match_count,page_count,volume_count,word_count
5879,! \t2008\t8\t8\t8\n! 0,1845.0,1.0,1.0,1.0,5.0
5895,"! 0 \t2008\t5\t5\t5\n! ? ? ? """"""",1919.0,1.0,1.0,1.0,5.0
9081,! It was God's will,1860.0,2.0,2.0,2.0,5.0
9082,! It was God's will,1863.0,2.0,2.0,2.0,5.0
9083,! It was God's will,1866.0,2.0,2.0,2.0,5.0


Precisamos remover itens com grafia estranha (ex: ! \t2008\t8\t8\t8\n! 0)?
Não, pois temos poucos registros com baixa ocorrência entre anos. 

In [17]:
df_keys = df.groupby('n_grams').count()

In [18]:
df_keys.head()

,year,match_count,page_count,volume_count,word_count
n_grams,,,,,
! \t2008\t8\t8\t8\n! 0,1,1,1,1,1
"! 0 \t2008\t5\t5\t5\n! ? ? ? """"""",1,1,1,1,1
! It was God's will,33,33,33,33,33
! It was the host's,26,26,26,26,26
! It's coming this way,22,22,22,22,22


In [19]:
df_keys.where(df_keys.word_count < 5).dropna().shape

(184, 5)

In [20]:
del df_keys

### Normalização de texto

In [21]:
df['n_grams'] = df['n_grams'].str.lower()

### Separando palavras em suas ordens

In [22]:
df['grams_array'] = df['n_grams'].apply(lambda sent: sent.split())

In [23]:
df['third_word'] = df['grams_array'].apply(lambda arr: arr[2])

In [24]:
df.head()

,n_grams,year,match_count,page_count,volume_count,word_count,grams_array,third_word
5879,! \t2008\t8\t8\t8\n! 0,1845.0,1.0,1.0,1.0,5.0,"[!, 2008, 8, 8, 8, !, 0]",8
5895,"! 0 \t2008\t5\t5\t5\n! ? ? ? """"""",1919.0,1.0,1.0,1.0,5.0,"[!, 0, 2008, 5, 5, 5, !, ?, ?, ?, """"""]",2008
9081,! it was god's will,1860.0,2.0,2.0,2.0,5.0,"[!, it, was, god's, will]",was
9082,! it was god's will,1863.0,2.0,2.0,2.0,5.0,"[!, it, was, god's, will]",was
9083,! it was god's will,1866.0,2.0,2.0,2.0,5.0,"[!, it, was, god's, will]",was


In [25]:
#Remove a terceira palavra do grams_array
df['grams_array'] = df['grams_array'].apply(lambda arr: arr[0:2]+arr[3:5])

In [26]:
df.head()

,n_grams,year,match_count,page_count,volume_count,word_count,grams_array,third_word
5879,! \t2008\t8\t8\t8\n! 0,1845.0,1.0,1.0,1.0,5.0,"[!, 2008, 8, 8]",8
5895,"! 0 \t2008\t5\t5\t5\n! ? ? ? """"""",1919.0,1.0,1.0,1.0,5.0,"[!, 0, 5, 5]",2008
9081,! it was god's will,1860.0,2.0,2.0,2.0,5.0,"[!, it, god's, will]",was
9082,! it was god's will,1863.0,2.0,2.0,2.0,5.0,"[!, it, god's, will]",was
9083,! it was god's will,1866.0,2.0,2.0,2.0,5.0,"[!, it, god's, will]",was


In [27]:
#df.groupby(df['third_word'])['third_word'].count().sort_values(ascending=False)[60:80]

### Seleção dos termos de interesse (woman e man)

In [28]:
df_analysis = df[(df['third_word'] =='man') | (df['third_word'] =='woman')].copy().dropna()

In [29]:
df_analysis.groupby('third_word').count()

,n_grams,year,match_count,page_count,volume_count,word_count,grams_array
third_word,,,,,,,
man,8763,8763,8763,8763,8763,8763,8763
woman,1830,1830,1830,1830,1830,1830,1830


In [30]:
df_analysis.head()

,n_grams,year,match_count,page_count,volume_count,word_count,grams_array,third_word
2925954,a certain woman who had,1857.0,1.0,1.0,1.0,5.0,"[a, certain, who, had]",woman
2925955,a certain woman who had,1859.0,1.0,1.0,1.0,5.0,"[a, certain, who, had]",woman
2925956,a certain woman who had,1861.0,1.0,1.0,1.0,5.0,"[a, certain, who, had]",woman
2925957,a certain woman who had,1876.0,3.0,3.0,3.0,5.0,"[a, certain, who, had]",woman
2925958,a certain woman who had,1879.0,3.0,3.0,3.0,5.0,"[a, certain, who, had]",woman


### Remoção de stopwords

In [31]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jairo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
df_analysis['grams_array'] =  df_analysis['grams_array'].apply(lambda sent: [word for word in sent if (not word in stops)])

In [33]:
df_analysis.iloc[150:160]

,n_grams,year,match_count,page_count,volume_count,word_count,grams_array,third_word
3151891,de nigger woman is de,1975.0,2.0,2.0,1.0,5.0,"[de, nigger, de]",woman
3151892,de nigger woman is de,1978.0,4.0,4.0,4.0,5.0,"[de, nigger, de]",woman
3151893,de nigger woman is de,1980.0,1.0,1.0,1.0,5.0,"[de, nigger, de]",woman
3151894,de nigger woman is de,1982.0,3.0,3.0,3.0,5.0,"[de, nigger, de]",woman
3151895,de nigger woman is de,1983.0,2.0,2.0,2.0,5.0,"[de, nigger, de]",woman
3151896,de nigger woman is de,1984.0,2.0,2.0,2.0,5.0,"[de, nigger, de]",woman
3151897,de nigger woman is de,1985.0,2.0,2.0,2.0,5.0,"[de, nigger, de]",woman
3151898,de nigger woman is de,1986.0,6.0,6.0,6.0,5.0,"[de, nigger, de]",woman
3151899,de nigger woman is de,1987.0,10.0,10.0,10.0,5.0,"[de, nigger, de]",woman
3151900,de nigger woman is de,1988.0,9.0,9.0,9.0,5.0,"[de, nigger, de]",woman


In [34]:
vocabulary = []
for i in list(df_analysis.index):
    grams = df_analysis['grams_array'].loc[i]
    for word in grams:
        if not word in vocabulary:
            vocabulary.append(word)

In [35]:
len(vocabulary)

229

        w1   w2   w3   w4   
w1 1995 1    2    21   2
w1 1996 1    2    21   2
w1 1997 1    2    21   2
w1 1998 1    2    21   2
w2 1995 1    2    21   2
w2 1996 1    2    21   2
w2 1997 1    2    21   2
w2 1998 1    2    21   2

In [36]:
result =  {}
for third_word in df_analysis['third_word'].unique():
    subset_third_word = df_analysis[df_analysis['third_word']==third_word].dropna()
    for n_grams in subset_third_word['n_grams'].unique():
        subset_n_grams = subset_third_word[subset_third_word['n_grams']==n_grams].dropna()
        for year in subset_n_grams['year'].unique():
            subset_year = subset_n_grams[subset_n_grams['year']==year]
            words = subset_year['grams_array'].to_list()[0]
            for word in words:
                if not result.get(third_word):
                    result[third_word] = {}
                    result[third_word][year] = {}
                    result[third_word][year][word] = {'match_count': subset_year['match_count'].values[0],
                                                      'volume_count': subset_year['volume_count'].values[0]}
                else:
                    if not result[third_word].get(year):
                        result[third_word][year] = {}
                        result[third_word][year][word] = {'match_count': subset_year['match_count'].values[0],
                                                          'volume_count': subset_year['volume_count'].values[0]}
                    else:
                        if not result[third_word][year].get(word):
                            result[third_word][year][word] = {'match_count': subset_year['match_count'].values[0],
                                                          'volume_count': subset_year['volume_count'].values[0]}      
                        else:
                            result[third_word][year][word]['match_count'] += subset_year['match_count'].values[0]
                            result[third_word][year][word]['volume_count'] += subset_year['volume_count'].values[0]

In [37]:
result_match_count =  {}
result_volume_count =  {}
for third_word in df_analysis['third_word'].unique():
    subset_third_word = df_analysis[df_analysis['third_word']==third_word].dropna()
    for n_grams in subset_third_word['n_grams'].unique():
        subset_n_grams = subset_third_word[subset_third_word['n_grams']==n_grams].dropna()
        for year in subset_n_grams['year'].unique():
            year = int(year)
            subset_year = subset_n_grams[subset_n_grams['year']==year]
            words = subset_year['grams_array'].to_list()[0]
            for word in words:
                if not result_match_count.get(third_word):
                    result_match_count[third_word] = {}
                    result_match_count[third_word][year] = {}
                    result_match_count[third_word][year][word] = subset_year['match_count'].values[0]
                    
                    result_volume_count[third_word] = {}
                    result_volume_count[third_word][year] = {}
                    result_volume_count[third_word][year][word] = subset_year['volume_count'].values[0]
                else:
                    if not result_match_count[third_word].get(year):
                        result_match_count[third_word][year] = {}
                        result_match_count[third_word][year][word] = subset_year['match_count'].values[0]
                        
                        result_volume_count[third_word][year] = {}
                        result_volume_count[third_word][year][word] = subset_year['volume_count'].values[0]
                    else:
                        if not result_match_count[third_word][year].get(word):
                            result_match_count[third_word][year][word] = subset_year['match_count'].values[0]
                            result_volume_count[third_word][year][word] = subset_year['volume_count'].values[0]
                        else:
                            #print(result_match_count[third_word][year])
                            #print(word)
                            #print(subset_year['match_count'].values[0])
                            result_match_count[third_word][year][word] += subset_year['match_count'].values[0]
                            result_volume_count[third_word][year][word] += subset_year['volume_count'].values[0]

In [38]:
result_volume_count['woman'][1865]

{'like': 1.0, 'counting': 1.0, 'told': 1.0, 'aspire': 1.0}

In [39]:
df_mc_woman = pd.DataFrame(result_match_count['woman'])
df_mc_man = pd.DataFrame(result_match_count['man'])
df_mc_woman['third_word'] = 'woman'
df_mc_man['third_word'] = 'man'
df_mc_woman['count'] = 'match_count'
df_mc_man['count'] = 'match_count'

df_vc_woman = pd.DataFrame(result_volume_count['woman'])
df_vc_man = pd.DataFrame(result_volume_count['man'])
df_vc_woman['third_word'] = 'woman'
df_vc_man['third_word'] = 'man'
df_vc_woman['count'] = 'volume_count'
df_vc_man['count'] = 'volume_count'

df_result = pd.concat([df_mc_man,df_mc_woman,df_vc_man,df_vc_woman])
df_result.reset_index(level=0, inplace=True)

C:\Users\Jairo\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [40]:
df_result.head()

,index,1759,1813,1846,1847,1850,1851,1853,1859,1860,...,1852,1858,1866,1870,1871,1872,1876,1879,third_word,count
0,!,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,man,match_count
1,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,man,match_count
2,:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,man,match_count
3,according,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,man,match_count
4,acquainted,NaN,NaN,1.0,12.0,1.0,NaN,NaN,4.0,8.0,...,7.0,2.0,1.0,1.0,1.0,NaN,3.0,NaN,man,match_count


In [41]:
df_result.columns = [str(name) for name in df_result.columns]

In [42]:
match_counts = df_result[df_result['count']=='match_count'].dropna(subset=['count']).iloc[:,1:232]
print(match_counts.shape)
df_tf = match_counts.div(match_counts.sum())
print(df_tf.shape)
df_tf.head()

(240, 231)
(240, 231)


,1759,1813,1846,1847,1850,1851,1853,1859,1860,1863,...,1839,1840,1852,1858,1866,1870,1871,1872,1876,1879
0,NaN,NaN,NaN,NaN,NaN,NaN,0.021505,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009091,0.006211,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.021505,NaN,NaN,NaN,...,0.021277,NaN,NaN,NaN,NaN,0.016529,NaN,NaN,0.007752,NaN
4,NaN,NaN,0.009091,0.086331,0.011364,NaN,NaN,0.036364,0.049689,NaN,...,0.063830,0.060976,0.058333,0.026667,0.008929,0.008264,0.010101,NaN,0.023256,NaN


In [43]:
print(df_tf.shape)

(240, 231)


In [44]:
volume_counts = df_result[df_result['count']=='volume_count'].dropna(subset=['count']).iloc[:,1:232]

df_idf = volume_counts.div(volume_counts.sum())**-1
df_idf.head(5)

,1759,1813,1846,1847,1850,1851,1853,1859,1860,1863,...,1839,1840,1852,1858,1866,1870,1871,1872,1876,1879
240,NaN,NaN,NaN,NaN,NaN,NaN,46.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.0,147.000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,46.5,NaN,NaN,NaN,...,46.000000,NaN,NaN,NaN,NaN,60.0,NaN,NaN,119.000000,NaN
244,NaN,NaN,99.0,11.416667,85.0,NaN,NaN,27.5,18.375,NaN,...,15.333333,16.4,16.285714,36.5,106.0,120.0,99.0,NaN,39.666667,NaN


In [45]:
#CONSIDERAR W
df_result[df_result['count']=='match_count'].dropna(subset=['count']).iloc[:,1:232].notna().any(axis=1).sum()

240

In [46]:
volumes_per_year = match_counts.sum()

In [47]:
df_tfidf = df_tf*df_idf.reset_index(drop=True).apply(np.log)
df_tfidf.head()

,1759,1813,1846,1847,1850,1851,1853,1859,1860,1863,...,1839,1840,1852,1858,1866,1870,1871,1872,1876,1879
0,NaN,NaN,NaN,NaN,NaN,NaN,0.082569,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042732,0.030996,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0.082569,NaN,NaN,NaN,...,0.081460,NaN,NaN,NaN,NaN,0.067675,NaN,NaN,0.037047,NaN
4,NaN,NaN,0.041774,0.210222,0.050485,NaN,NaN,0.120516,0.144646,NaN,...,0.174257,0.170566,0.162767,0.095928,0.041638,0.039566,0.046415,NaN,0.085593,NaN


In [48]:
#checking
#df_tf.loc[4,1846]*np.log(df_idf.reset_index(drop=True).loc[4,1846])

TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [1846] of <class 'int'>

In [49]:
woman_over_time = df_result[(df_result['third_word']=='woman') & (df_result['count']=='match_count')].dropna(subset=['third_word', 'count'])
woman_over_time.drop(['third_word', 'count'], axis=1, inplace=True)

In [50]:
woman_over_time = (woman_over_time.iloc[:,1:].reindex(sorted(woman_over_time.iloc[:,1:].columns), axis=1)
                   .merge(woman_over_time['index'], left_index=True, right_index=True))

In [51]:
woman_top_word_time = woman_over_time.iloc[:,:-1].idxmax(axis=0).dropna().map(woman_over_time['index'])

In [52]:
woman_top_word_time.to_excel('woman_top_word_time.xls')

In [53]:
man_over_time = df_result[(df_result['third_word']=='man') & (df_result['count']=='match_count')].dropna(subset=['third_word', 'count'])
man_over_time.drop(['third_word', 'count'], axis=1, inplace=True)

man_over_time = (man_over_time.iloc[:,1:].reindex(sorted(man_over_time.iloc[:,1:].columns), axis=1)
                   .merge(man_over_time['index'], left_index=True, right_index=True))

man_top_word_time = man_over_time.iloc[:,:-1].idxmax(axis=0).dropna().map(man_over_time['index'])

man_top_word_time.to_excel('man_top_word_time.xls')

In [ ]:
man_over_time = df_result[(df_result['third_word']=='man') & (df_result['count']=='match_count')].dropna(subset=['third_word', 'count'])
man_over_time.drop(['third_word', 'count'], axis=1, inplace=True)

man_over_time = (man_over_time.iloc[:,1:].reindex(sorted(man_over_time.iloc[:,1:].columns), axis=1)
                   .merge(man_over_time['index'], left_index=True, right_index=True))

man_top_word_time = man_over_time.iloc[:,:-1].idxmax(axis=0).dropna().map(man_over_time['index'])

man_top_word_time.to_excel('man_top_word_time.xls')

In [ ]:
df_tfidf